<a href="https://colab.research.google.com/github/vsa1920/Football-Data/blob/main/Final_Match_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from tensorflow import keras
from sklearn.metrics import classification_report, accuracy_score


In [64]:
player_data = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_players.csv")
match_data = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_matches_train.csv")
potential_shots = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_potential_shots.csv")
goals = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_goals.csv")
match_test = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_matches_test.csv")
player_data.drop(["birthday", "height", "weight", "preferred_foot", "attacking_work_rate", "defensive_work_rate"], axis=1, inplace=True)
player_data = player_data.dropna()
match_data = match_data.dropna()
match_data.head()

,season,stage,date,match_id,home_team_id,away_team_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,...,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,on_target_shot_home_team,on_target_shot_away_team,off_target_shot_home_team,off_target_shot_away_team,foul_home_team,foul_away_team,yellow_card_home_team,yellow_card_away_team,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team,home_team_goal,away_team_goal
0,2008/2009,1,2008-08-17 00:00:00,49337,10260,10261,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,5,5,1,3,3,3,3,7,7,7,7,10,10,1,...,30726,30362.0,30620,30865,32569.0,24148,34944.0,30373.0,24154.0,24157.0,30829.0,24224,25518.0,24228.0,30929,29581.0,38807.0,40565.0,30360.0,33852.0,34574.0,37799.0,11,1,10,9,16,11,3,0,0,0,24,9,6,6,55.0,45.0,1,1
1,2008/2009,1,2008-08-16 00:00:00,38136,9825,8659,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,5,7,9,1,3,5,1,3,3,3,3,7,7,7,7,10,10,1,...,23686,26111.0,38835,30986,31291.0,31013,30935.0,39297.0,26181.0,30960.0,36410.0,36373,36832.0,23115.0,37280,24728.0,24664.0,31088.0,23257.0,24171.0,25922.0,27267.0,12,2,13,3,11,9,0,0,0,0,21,7,7,5,66.0,34.0,1,0
2,2008/2009,1,2008-08-16 00:00:00,43276,8472,8650,1,2,4,6,8,2,4,6,8,4,6,1,2,4,6,8,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,...,32562,38836.0,24446,24408,36786.0,38802,24655.0,17866.0,30352.0,23927.0,24410.0,30660,37442.0,30617.0,24134,414792.0,37139.0,30618.0,40701.0,24800.0,24635.0,30853.0,4,11,3,5,13,12,0,2,0,0,15,19,1,8,46.0,54.0,0,1
3,2008/2009,1,2008-08-16 00:00:00,40671,8654,8528,1,2,4,6,8,2,4,6,8,4,6,1,2,6,8,4,2,4,6,8,4,6,1,3,3,3,3,7,7,7,7,10,10,1,...,36374,30966.0,23818,37277,30687.0,36394,37169.0,24223.0,24773.0,34543.0,23139.0,34421,34987.0,35472.0,111865,25005.0,35327.0,25150.0,97988.0,41877.0,127857.0,34466.0,5,7,7,15,14,13,2,1,0,0,15,27,6,10,52.0,48.0,2,1
5,2008/2009,1,2008-08-16 00:00:00,45612,8668,8655,1,2,4,6,8,1,3,5,7,9,5,1,2,4,6,8,4,6,8,2,6,4,1,3,3,3,3,7,7,7,7,7,11,1,...,31465,30371.0,24004,33086,30857.0,24011,109058.0,23268.0,24846.0,24006.0,24160.0,30622,37764.0,19020.0,23921,24136.0,30342.0,23889.0,23916.0,23922.0,34176.0,30646.0,2,8,7,8,11,11,2,2,0,0,14,21,3,4,51.0,49.0,2,3


In [65]:
match_data["date"] = pd.to_datetime(match_data["date"])
player_data["date"] = pd.to_datetime(player_data["date"])
player_data.head()

,player_id,date,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,155782,2016-04-21,80.0,53.0,58.0,71.0,40.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,62.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
1,155782,2016-04-07,80.0,53.0,58.0,71.0,32.0,73.0,70.0,69.0,68.0,71.0,79.0,78.0,78.0,67.0,90.0,71.0,85.0,79.0,56.0,60.0,68.0,67.0,60.0,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
2,155782,2016-01-07,79.0,52.0,57.0,70.0,29.0,71.0,68.0,69.0,68.0,70.0,79.0,78.0,78.0,67.0,90.0,71.0,84.0,79.0,56.0,59.0,67.0,66.0,58.0,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
3,155782,2015-12-24,79.0,51.0,57.0,70.0,29.0,71.0,68.0,69.0,68.0,70.0,79.0,78.0,78.0,67.0,90.0,71.0,84.0,79.0,56.0,58.0,67.0,66.0,58.0,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
4,155782,2015-12-17,79.0,51.0,57.0,70.0,29.0,71.0,68.0,69.0,68.0,70.0,79.0,78.0,78.0,67.0,90.0,71.0,84.0,79.0,56.0,58.0,67.0,66.0,58.0,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [66]:
def preprocess_X(row):
    """
    Generate each row of data for each match.
    """
    X = []
    date = row[2]
    for positions in row[6:50]:
      X.append(positions)
    for player in row[50:72]:
      X.append(player)
      p_data = player_data.loc[player_data["player_id"] == player]
      pt_data = p_data.loc[p_data["date"] < date]
      pt_data.sort_values(by=["date"])
      try:
        X += list(pt_data.iloc[-1,2:])
      except:
        return None
    return X

In [67]:
def decision(home_goals, away_goals):
  if home_goals > away_goals:
    return 0
  elif home_goals == away_goals:
    return 1
  else:
    return 2

In [68]:
X_data = []
y_data = []

for index, row in match_data.iterrows():
  x_data = preprocess_X(row)
  if x_data == None:
    continue
  else:
    X_data.append(x_data)
  y_data.append(decision(row['home_team_goal'], row['away_team_goal']))

Split the dataset into training, validation and test sets:-

In [70]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_data, y_data, random_state=100, test_size=0.1)

In [71]:
train_X, val_X, train_y, val_y = train_test_split(np.array(X_train_val), np.array(y_train_val), random_state=100, test_size=0.1)

In [72]:
# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=0)

In [73]:
rf_clf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [74]:
# validation performance
print (accuracy_score(val_y, rf_clf.predict(val_X)))

0.5518867924528302


In [75]:
# test performance
print (accuracy_score(y_test, rf_clf.predict(X_test)))

0.5


In [76]:
# A classification report of test performance
print (classification_report(y_test, rf_clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.56      0.84      0.67       112
           1       0.17      0.02      0.03        63
           2       0.37      0.38      0.37        61

    accuracy                           0.50       236
   macro avg       0.36      0.41      0.36       236
weighted avg       0.41      0.50      0.42       236



In [77]:
input_dim = train_X.shape[1]
input_dim

792

In [31]:
model1 = keras.Sequential([
                          keras.layers.Dense(256, activation='relu', input_shape=[input_dim], kernel_regularizer=keras.regularizers.l2(1)),
                          keras.layers.BatchNormalization(),
                          #keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.1)),
                          #keras.layers.BatchNormalization(),
                          keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(1)),
                          keras.layers.BatchNormalization(),
                          #keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.1)),
                          #keras.layers.BatchNormalization(),
                          keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(1)),
                          keras.layers.Dense(3, activation='softmax')
])

In [45]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
opt = keras.optimizers.Adam(learning_rate=0.0001)
model1.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor="val_accuracy")

In [46]:
model1.fit(x=train_X, y=train_y, epochs=100, batch_size=512, validation_data=(val_X, val_y),)

Epoch 1/5
4/4 [==============================] - 1s 88ms/step - loss: 429.7502 - accuracy: 0.3752 - val_loss: 424.8082 - val_accuracy: 0.3396
Epoch 2/5
4/4 [==============================] - 0s 29ms/step - loss: 423.4506 - accuracy: 0.3695 - val_loss: 418.5788 - val_accuracy: 0.3491
Epoch 3/5
4/4 [==============================] - 0s 29ms/step - loss: 417.2264 - accuracy: 0.3936 - val_loss: 412.4276 - val_accuracy: 0.3491
Epoch 4/5
4/4 [==============================] - 0s 30ms/step - loss: 411.0996 - accuracy: 0.3768 - val_loss: 406.3575 - val_accuracy: 0.3396
Epoch 5/5
4/4 [==============================] - 0s 28ms/step - loss: 405.0426 - accuracy: 0.3950 - val_loss: 400.3701 - val_accuracy: 0.3538


In [80]:
match_id = []
X_test = []
for index, row in match_test.iterrows():
  x_data = preprocess_X(row)
  if x_data == None:
    continue
  else:
    X_test.append(x_data)
    match_id.append(row["match_id"])

In [78]:
out_dict = {0: "win", 1: "tie", 2: "loss"}

In [90]:
pred = [out_dict[p] for p in rf_clf.predict(X_test)]
predictions_frame = pd.DataFrame(list(zip(match_id, pred)), columns=["match_id", "result"])

<class 'list'> 377


In [94]:
predictions_frame.to_csv("prediction_submission.csv")
from google.colab import files
files.download("prediction_submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>